In [1]:
# importing libraries here to better keep track of them

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Reading csv files into memory

injury_history_raw = pd.read_csv("data/injury_history(injury_history).csv", sep = ",", encoding = 'ISO-8859-1')
muscle_imbalance_raw = pd.read_csv("data/injury_history(muscle_imbalance_data).csv", sep = ",", encoding = 'ISO-8859-1')
player_sessions_raw = pd.read_csv("data/injury_history(player_sessions).csv", sep = ",", encoding = 'ISO-8859-1')

In [3]:
player_sessions_raw['Month'] = pd.to_datetime(player_sessions_raw['Session_Date']).dt.month
muscle_imbalance_raw['Month'] = pd.to_datetime(muscle_imbalance_raw['Date Recorded']).dt.month

In [4]:
print(f"Raw Injury Data: \n {injury_history_raw.head()} \n")
print(f"Raw Muscle Imbalance Data: \n {muscle_imbalance_raw.head()} \n")
print(f"Raw Player Session Data: \n {player_sessions_raw.head()} \n")

Raw Injury Data: 
    Player.ID             Name  Group.Id    Injury Type   Body Part   Side  \
0        101  Jordan Matthews       201  Muscle Strain  Quadriceps  Right   
1        101  Jordan Matthews       201     Tendonitis       Wrist   Left   
2        101  Jordan Matthews       201     Tendonitis    Shoulder  Right   
3        103   Malik Robinson       203         Strain       Groin  Right   
4        103   Malik Robinson       203       Fracture       Wrist   Left   

  Injury Date Severity  Recovery Time (days)  \
0   12/5/2023  Grade 2                    51   
1  10/25/2023      NaN                    11   
2   7/22/2023      NaN                    12   
3   6/28/2023  Grade 1                    20   
4   2/14/2023      NaN                    68   

                                    Additional Notes  
0  Grade 2 quadriceps strain with partial tearing...  
1  De Quervain's tenosynovitis. Swelling and pain...  
2  Rotator cuff tendonitis due to overuse. Anti-i...  
3  Grade 

In [5]:
print(f'Columns present in injury data: \n {injury_history_raw.columns} \n')
print(f'Columns present in muscle imbalance data: \n {muscle_imbalance_raw.columns} \n')
print(f'Columns present in player session data: \n {player_sessions_raw.columns} \n')

Columns present in injury data: 
 Index(['Player.ID', 'Name', 'Group.Id', 'Injury Type', 'Body Part', 'Side',
       'Injury Date', 'Severity', 'Recovery Time (days)', 'Additional Notes'],
      dtype='object') 

Columns present in muscle imbalance data: 
 Index(['Player.ID', 'Session ID', 'Player Name', 'Date Recorded',
       'Hamstring To Quad Ratio', 'Quad Imbalance Percent',
       'HamstringImbalance Percent', 'Calf Imbalance Percent',
       'Groin Imbalance Percent', 'Month'],
      dtype='object') 

Columns present in player session data: 
 Index(['Name', 'Player.ID', 'Group.Id', 'Group.name', 'League.ID',
       'Session.ID', 'Session_Date', 'Position', 'Distance..mi.',
       'Distance...min..mi.', 'Duration..s.', 'Steps', 'Speed....of.max......',
       'Speed..max....mph.', 'Speed..?ò...mph.', 'Time..s.',
       'Accumulated.Acceleration.Load', 'Anaerobic.Activity..distance...mi.',
       'Jump.Load..J.', 'Heart.Rate..?ò...bpm.', 'Heart.Rate..min....bpm.',
       'Heart.Ra

## Player Session Data column research

#### TRIMP - A way to measure Training Impulse (Aerobic Training Load). It is a product of Training volume (minutes) and Training intensity (beats per minutes). 

TRIMP = Training VOlume * Training Intensity. For example if a workout is 50 minutes and the average heart rate (bpm) is 140 bpm, TRIMP score is calculated out to 7000. TRIMP is affected by a number of factors but we can look for correlation on a correlation matrix as well. 
    - Physical Settings
    - Resting and Maximal Heart Rate
    - Gender (since its mbb there's not going to be any variance with this feature)

TRIMP can be used to compare sessions of different lengths, or to compare high-intensity sessions with longer game data. For example if a session with TRIMP score of 108 might be considered moderate, the TRIMP/min score could indicate an intense session if Training Volume is lower. Training Stress Score (TSS) is a proprietary variant based on a score of 100 for a 1-hour maximum sustained effort. 
Positive correlation between Heart Rate and Effort

---

In [6]:
print(f'Size of injury history data: \n {injury_history_raw.shape}\n')
print(f'Size of muscle imbalance data: \n {muscle_imbalance_raw.shape}\n')
print(f'Size of player session data: \n {player_sessions_raw.shape}\n')

Size of injury history data: 
 (21, 10)

Size of muscle imbalance data: 
 (182, 10)

Size of player session data: 
 (2604, 31)



## Null Values

In [7]:
print(f'Null Values in injury history data: \n {injury_history_raw.isnull().sum()}')
print(f'Null Values in muscle imbalance data: \n {muscle_imbalance_raw.isnull().sum()}')
print(f'Null Values in player session data: \n {player_sessions_raw.isnull().sum()}')

Null Values in injury history data: 
 Player.ID                0
Name                     0
Group.Id                 0
Injury Type              0
Body Part                0
Side                     5
Injury Date              0
Severity                10
Recovery Time (days)     0
Additional Notes         0
dtype: int64
Null Values in muscle imbalance data: 
 Player.ID                     0
Session ID                    0
Player Name                   0
Date Recorded                 0
Hamstring To Quad Ratio       0
Quad Imbalance Percent        0
HamstringImbalance Percent    0
Calf Imbalance Percent        0
Groin Imbalance Percent       0
Month                         0
dtype: int64
Null Values in player session data: 
 Name                                  0
Player.ID                             0
Group.Id                              0
Group.name                            0
League.ID                             0
Session.ID                            0
Session_Date                

Okay so with null values, injury history data is the only dataset with null values. There are 2 columns with null values with a total of 15 null values. Less than 10% of the data. We should delete the data since we are working with health data instead of imputing the values. I think it will introduce bias through assumption. Our train test split is going to be pretty weak since we only will have 167 instances. 

In [19]:
twothird_data = pd.merge(muscle_imbalance_raw, player_sessions_raw, on = ['Player.ID', 'Month'], how = 'right')
twothird_data.head()

,Player.ID,Session ID,Player Name,Date Recorded,Hamstring To Quad Ratio,Quad Imbalance Percent,HamstringImbalance Percent,Calf Imbalance Percent,Groin Imbalance Percent,Month,...,Heart.Rate..min....bpm.,Heart.Rate..max....bpm.,Human.Core.Temperature..?ò....F.,Human.Core.Temperature..max.....F.,TRIMP,Heart.Rate.Recoveries,Jump.Height..max....ft.,Changes.of.Orientation,Exertions,Disk.Usage....
0,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,1,...,74,198,99.47,101.24,261,5,2.31,229,307,58.56
1,112,113,Anthony Lopez,1/1/2024,0.919842,-10.238526,-9.417831,-10.206984,-10.126625,1,...,74,198,99.47,101.24,261,5,2.31,229,307,58.56
2,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,1,...,62,179,99.56,99.33,270,6,2.44,427,180,44.93
3,112,113,Anthony Lopez,1/1/2024,0.919842,-10.238526,-9.417831,-10.206984,-10.126625,1,...,62,179,99.56,99.33,270,6,2.44,427,180,44.93
4,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,1,...,78,172,100.06,102.31,149,4,3.04,383,440,15.32


In [9]:
obt = pd.merge(twothird_data, injury_history_raw, on = 'Player.ID', how = 'left')
obt.tail()

,Player.ID,Session ID,Player Name,Date Recorded,Hamstring To Quad Ratio,Quad Imbalance Percent,HamstringImbalance Percent,Calf Imbalance Percent,Groin Imbalance Percent,Month,...,Disk.Usage....,Name_y,Group.Id_y,Injury Type,Body Part,Side,Injury Date,Severity,Recovery Time (days),Additional Notes
4716,108,112,Xavier Foster,12/1/2023,1.106029,-8.500787,-9.402117,-8.825422,-8.92167,12,...,2.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4717,108,112,Xavier Foster,12/1/2023,1.106029,-8.500787,-9.402117,-8.825422,-8.92167,12,...,87.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4718,108,112,Xavier Foster,12/1/2023,1.106029,-8.500787,-9.402117,-8.825422,-8.92167,12,...,36.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4719,108,112,Xavier Foster,12/1/2023,1.106029,-8.500787,-9.402117,-8.825422,-8.92167,12,...,53.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4720,108,112,Xavier Foster,12/1/2023,1.106029,-8.500787,-9.402117,-8.825422,-8.92167,12,...,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
obt.isnull().sum()

Player.ID                                0
Session ID                               0
Player Name                              0
Date Recorded                            0
Hamstring To Quad Ratio                  0
Quad Imbalance Percent                   0
HamstringImbalance Percent               0
Calf Imbalance Percent                   0
Groin Imbalance Percent                  0
Month                                    0
Name_x                                   0
Group.Id_x                               0
Group.name                               0
League.ID                                0
Session.ID                               0
Session_Date                             0
Position                                 0
Distance..mi.                            0
Distance...min..mi.                      0
Duration..s.                             0
Steps                                    0
Speed....of.max......                    0
Speed..max....mph.                       0
Speed..?ò..

In [11]:
obt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4721 entries, 0 to 4720
Data columns (total 48 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Player.ID                           4721 non-null   int64  
 1   Session ID                          4721 non-null   int64  
 2   Player Name                         4721 non-null   object 
 3   Date Recorded                       4721 non-null   object 
 4   Hamstring To Quad Ratio             4721 non-null   float64
 5   Quad Imbalance Percent              4721 non-null   float64
 6   HamstringImbalance Percent          4721 non-null   float64
 7   Calf Imbalance Percent              4721 non-null   float64
 8   Groin Imbalance Percent             4721 non-null   float64
 9   Month                               4721 non-null   int32  
 10  Name_x                              4721 non-null   object 
 11  Group.Id_x                          4721 no

In [13]:
# Dropping columns that are repeated and redundant (no variance among values in column). 
distinct_obt = obt.drop(columns = ['Group.name', 'League.ID', 'Name_y', 'Group.Id_y', 'Name_x'])

In [17]:
distinct_obt.isna().sum()

Player.ID                                0
Session ID                               0
Player Name                              0
Date Recorded                            0
Hamstring To Quad Ratio                  0
Quad Imbalance Percent                   0
HamstringImbalance Percent               0
Calf Imbalance Percent                   0
Groin Imbalance Percent                  0
Month                                    0
Group.Id_x                               0
Session.ID                               0
Session_Date                             0
Position                                 0
Distance..mi.                            0
Distance...min..mi.                      0
Duration..s.                             0
Steps                                    0
Speed....of.max......                    0
Speed..max....mph.                       0
Speed..?ò...mph.                         0
Time..s.                                 0
Accumulated.Acceleration.Load            0
Anaerobic.A

In [20]:
# Injury type, Body Part, Injury Date, Recovery Time, and Additional Notes all have the same number of null values indicating that these players may not have injuries. 
distinct_obt['Injury Type'].fillna("Not Injured", inplace=True)
distinct_obt['Body Part'].fillna("None", inplace = True)
distinct_obt['Injury Date'].fillna("0/0/0000", inplace = True)
distinct_obt['Recovery Time (days)'].fillna(0, inplace = True) 
distinct_obt['Additional Notes'].fillna("Not Injured", inplace = True)

/tmp/ipykernel_2681/2470867868.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  distinct_obt['Recovery Time (days)'].fillna(0, inplace = True)
/tmp/ipykernel_2681/2470867868.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [ ]:
distinct_obt.loc[distinct_obt['Injury Type'] == "Not Injured", 'Severity'] = distinct_obt.loc[distinct_obt['Injury Type'] == "Not Injured", 'Severity'].fillna("Grade 0")
distinct_obt.loc[distinct_obt['Injury Type'] == "Not Injured", 'Side'] = distinct_obt.loc[distinct_obt['Injury Type'] == "Not Injured", 'Side'].fillna("No Injury")

In [36]:
clean_obt = distinct_obt.dropna()
clean_obt.head()

,Player.ID,Session ID,Player Name,Date Recorded,Hamstring To Quad Ratio,Quad Imbalance Percent,HamstringImbalance Percent,Calf Imbalance Percent,Groin Imbalance Percent,Month,...,Changes.of.Orientation,Exertions,Disk.Usage....,Injury Type,Body Part,Side,Injury Date,Severity,Recovery Time (days),Additional Notes
0,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,1,...,229,307,58.56,Strain,Knee,Left,1/26/2023,Grade 1,28.0,Strain of the posterior cruciate ligament (PCL...
1,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,1,...,229,307,58.56,Muscle Strain,Quadriceps,Left,11/18/2023,Grade 1,12.0,Grade 1 quadriceps strain. Minor tear in the m...
3,112,113,Anthony Lopez,1/1/2024,0.919842,-10.238526,-9.417831,-10.206984,-10.126625,1,...,229,307,58.56,Strain,Knee,Left,1/26/2023,Grade 1,28.0,Strain of the posterior cruciate ligament (PCL...
4,112,113,Anthony Lopez,1/1/2024,0.919842,-10.238526,-9.417831,-10.206984,-10.126625,1,...,229,307,58.56,Muscle Strain,Quadriceps,Left,11/18/2023,Grade 1,12.0,Grade 1 quadriceps strain. Minor tear in the m...
6,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,1,...,427,180,44.93,Strain,Knee,Left,1/26/2023,Grade 1,28.0,Strain of the posterior cruciate ligament (PCL...


In [38]:
clean_sorted_obt = clean_obt.sort_values(['Player.ID', 'Session ID', 'Date Recorded', 'Session.ID', 'Session_Date', 'Injury Date'])
clean_sorted_obt.head()

,Player.ID,Session ID,Player Name,Date Recorded,Hamstring To Quad Ratio,Quad Imbalance Percent,HamstringImbalance Percent,Calf Imbalance Percent,Groin Imbalance Percent,Month,...,Changes.of.Orientation,Exertions,Disk.Usage....,Injury Type,Body Part,Side,Injury Date,Severity,Recovery Time (days),Additional Notes
1605,101,101,Jordan Matthews,1/1/2023,0.610768,5.481295,3.347801,5.409497,5.616962,1,...,308,132,23.12,Muscle Strain,Quadriceps,Right,12/5/2023,Grade 2,51.0,Grade 2 quadriceps strain with partial tearing...
1611,101,101,Jordan Matthews,1/1/2023,0.610768,5.481295,3.347801,5.409497,5.616962,1,...,188,194,74.68,Muscle Strain,Quadriceps,Right,12/5/2023,Grade 2,51.0,Grade 2 quadriceps strain with partial tearing...
1617,101,101,Jordan Matthews,1/1/2023,0.610768,5.481295,3.347801,5.409497,5.616962,1,...,181,413,52.18,Muscle Strain,Quadriceps,Right,12/5/2023,Grade 2,51.0,Grade 2 quadriceps strain with partial tearing...
1623,101,101,Jordan Matthews,1/1/2023,0.610768,5.481295,3.347801,5.409497,5.616962,1,...,115,359,37.75,Muscle Strain,Quadriceps,Right,12/5/2023,Grade 2,51.0,Grade 2 quadriceps strain with partial tearing...
1629,101,101,Jordan Matthews,1/1/2023,0.610768,5.481295,3.347801,5.409497,5.616962,1,...,156,405,62.07,Muscle Strain,Quadriceps,Right,12/5/2023,Grade 2,51.0,Grade 2 quadriceps strain with partial tearing...


In [52]:
clean_sorted_obt['Injury_status'] = 1  
clean_sorted_obt['Injury_status'] = clean_sorted_obt['Injury_status'].where(clean_sorted_obt['Injury Type'] == 'Not Injured', 0)

In [53]:
clean_sorted_obt.columns

Index(['Player.ID', 'Player Name', 'Date Recorded', 'Hamstring To Quad Ratio',
       'Calf Imbalance Percent', 'Groin Imbalance Percent', 'Month',
       'Session_Date', 'Position', 'Distance..mi.', 'Distance...min..mi.',
       'Duration..s.', 'Steps', 'Speed....of.max......', 'Speed..max....mph.',
       'Speed..?ò...mph.', 'Time..s.', 'Accumulated.Acceleration.Load',
       'Anaerobic.Activity..distance...mi.', 'Jump.Load..J.',
       'Heart.Rate..?ò...bpm.', 'Heart.Rate..min....bpm.',
       'Heart.Rate..max....bpm.', 'Human.Core.Temperature..?ò....F.',
       'Human.Core.Temperature..max.....F.', 'TRIMP', 'Heart.Rate.Recoveries',
       'Jump.Height..max....ft.', 'Changes.of.Orientation', 'Exertions',
       'Disk.Usage....', 'Injury Type', 'Body Part', 'Side', 'Injury Date',
       'Severity', 'Recovery Time (days)', 'Additional Notes',
       'Injury_status'],
      dtype='object')

In [ ]:
clean_sorted_obt['Date Recorded'] = pd.to_datetime(clean_sorted_obt['Date Recorded'])
clean_sorted_obt['Session_Date'] = pd.to_datetime(clean_sorted_obt['Session_Date'])

In [65]:
columns_to_drop = ['Quad Imbalance Percent', 'HamstringImbalance Percent', 'Group.Id_x', 'Session ID', 'Session.ID']
for column in columns_to_drop:
    if column in clean_sorted_obt.columns:
        clean_sorted_obt.drop(columns = column, inplace = True)

In [ ]:

num_obt = clean_sorted_obt.select_dtypes(include=['number'])
correlation_matrix = num_obt.corr()
round(correlation_matrix['Injury_status'], 4).sort_values()

Recovery Time (days)                 -0.6116
Hamstring To Quad Ratio              -0.2853
Player.ID                            -0.1939
Calf Imbalance Percent               -0.1622
Groin Imbalance Percent              -0.1560
Distance...min..mi.                  -0.0537
Speed....of.max......                -0.0346
Jump.Load..J.                        -0.0331
Human.Core.Temperature..max.....F.   -0.0255
Duration..s.                         -0.0233
Exertions                            -0.0152
Anaerobic.Activity..distance...mi.   -0.0065
Month                                -0.0013
Distance..mi.                         0.0020
Human.Core.Temperature..?ò....F.      0.0023
Speed..?ò...mph.                      0.0032
Heart.Rate..max....bpm.               0.0045
Disk.Usage....                        0.0048
Heart.Rate..min....bpm.               0.0132
Time..s.                              0.0140
Steps                                 0.0150
Speed..max....mph.                    0.0190
Heart.Rate

In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop = 'first')

non_cat_columns = clean_sorted_obt.select_dtypes(include=['number', 'datetime']).columns
non_cat_data = clean_sorted_obt[non_cat_columns]

cat_columns = set(clean_sorted_obt.columns - non_cat_columns)
cat_data = clean_sorted_obt[cat_columns]

ohe_cat_data = ohe.fit_transform(cat_data)

ohe_injury_data = pd.concat([non_cat_data, ohe_cat_data], axis = 1)